# Exercises

Try each one alone, then check and compare with the person next to you.

There are many ways to solve each, so if your approaches differ, explain it to each other!

In [9]:
!pip install pandas_datareader

In [7]:
import pandas as pd
import pandas_datareader as pdr # IF NECESSARY, from terminal: pip install pandas_datareader 
import datetime
import numpy as np

start = datetime.datetime(2017, 1, 1) # you can specify start and end dates this way
end = datetime.datetime(2021, 1, 27)
macro_df = pdr.data.DataReader(['GDP','CPIAUCSL','UNRATE'], 'fred', start, end)

# prof demo+vocab coverage (students should type out if possible)
#    vocab: df, rows, columns/vars (accessing/series), index, 
#    how to: access 1 var (note: series), access multiple vars
#            access col names, rename (some cols, all cols via set_axis(1))
#            access index
#    index (reset and set)

## Part 1

During class, I used this dataframe to go over [Pandas vocab](02b_pandasVocab), and we show how to
- access 1 variable (note: `pd` calls this a "series" object, which is a 1D object instead of a 2D object)
- access multiple vars
- access, print, and change column names
- access, print, reset, and set the index


- Q0: Do each of the four new golden rules for initial data exploration, from the lecture.
    - monthly data, 49 months, GDP is quarterly so 2/3 obs are blank/missing/NaN (not a number)
- Q1: What is the second series above?
    - CPI (inflation) for urban consumers (what about flyover country)
- Q2: What is the frequency of the series?
    - monthly, but gdp is qrtly
- Q3: What is the average ANNUAL GDP, based on the data?
    - ~20,630

In [ ]:
start = datetime.datetime(2017, 1, 1) # you can specify start and end dates this way
end = datetime.datetime(2021, 1, 27)
macro_df = pdr.data.DataReader(['GDP','CPIAUCSL','UNRATE'], 'fred', start, end)

- Q0: Do each of the four new golden rules for initial data exploration, from the lecture.
- Q1: What is the second series above?
- Q2: What is the frequency of the series?
- Q3: What is the average ANNUAL GDP, based on the data?

In [1]:
# do your work here
# import a famous dataset, seaborn nicely contains it out of the box!
import seaborn as sns 
iris = sns.load_dataset('iris') 

print(iris.head(),  '\n---')
print(iris.tail(),  '\n---')
print(iris.columns, '\n---')
print("The shape is: ",iris.shape, '\n---')
print("Info:",iris.info(), '\n---') # memory usage, name, dtype, and # of non-null obs (--> # of missing obs) per variable
print(iris.describe(), '\n---') # summary stats, and you can customize the list!
print(iris['species'].value_counts()[:10], '\n---')
print(iris['species'].nunique(), '\n---')

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa 
---
     sepal_length  sepal_width  petal_length  petal_width    species
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica 
---
Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object') 
---
The shape is:  (150, 5) 
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex

## Part 2

- Q4: Download the annual *real* gdp from 1960 to 2018 from FRED and compute the average annual percent change
- Q5: Compute the average gdp percent change within *each decade*


In [11]:
# do your work here'=
# If youa re looking at a df and think: "to solve this problem, I would like to do a loop"
# DON'T

# ==> groupby = "for each of these groups" ... do something

part2_df = pdr.data.DataReader(['GDPCA'], 'fred', 1960, 2018)
part2_df['real_gdp_pct'] = part2_df['GDPCA'].pctchange()
part2_df['real_gdp_pct'].mean()

# v1.3 - save the intermediate pct change as new variable
(
    # DL data
    pdr.data.DataReader(['GDPCA'], 'fred', 1960, 2018) 
    
    # create the pct change var and call it "gdp_pct_change"
    # but the "object" assign is working on has no name!
    # how shall we access it?
    .assign(gdp_pct_change= lambda x: x['GDPCA'].pct_change()  )
        
    # output mean value
    ['gdp_pct_change'].mean()

)

AttributeError: 'Series' object has no attribute 'pctchange'

In [12]:
(
    # DL data
    pdr.data.DataReader(['GDPCA'], 'fred', 1960, 2018)
    
    # create the gdp pct change variable
     .assign(gdp_pct_change= lambda x: x['GDPCA'].pct_change()  )
    
    # create the decade variable 
    # extract the years form the DATE / INDEX, drop the last digit,
    .reset_index() # turn the index into a variable 
    .assign(decade = lambda x: (x['DATE'].dt.year//10)*10)
    
    # .index.dt.year
    # take the average FOR EACH DECADE == > groupby
    .groupby('decade') # for each decade!
    ['gdp_pct_change'] # grab this variable
    .mean()

)

decade
1960    0.047426
1970    0.032352
1980    0.031240
1990    0.032220
2000    0.019099
2010    0.023165
Name: gdp_pct_change, dtype: float64

## Part 3

First, I'll do the work to load January data on unemployment, the Case-Shiller housing index, and median household income in three states (CA/MI/PA). 

Then, we'll answer some questions

````{tip}
Run this block yourself, line-by-line, and part-by-part to figure out what I'm doing.

For example, just run the first three lines to download the data, then run
```py
macro_data.resample('Y')
```
Try other arguments inside resample to see what works (and what it does) and what doesn't work. 

````

In [13]:
# LOAD DATA AND CONVERT TO ANNUAL

start = 1990 # pandas datareader can infer these are years
end = 2018
macro_data = pdr.data.DataReader(['CAUR','MIUR','PAUR', # unemployment 
                                  'LXXRSA','DEXRSA','WDXRSA', # case shiller index in LA, Detroit, DC (no PA  available!)
                                  'MEHOINUSCAA672N','MEHOINUSMIA672N','MEHOINUSPAA672N'], #  
                                 'fred', start, end)
macro_data = macro_data.resample('Y').first() # get's the first observation for each variable in a given year

# CLEAN UP THE FORMATING SOMEWHAT

macro_data.index = macro_data.index.year
print("\n\n DATA BEFORE FORMATTING: \n\n")
print(macro_data[:20]) # see how the data looks now? ugly variable names, but its an annual dataset at least
macro_data.columns=pd.MultiIndex.from_tuples([
    ('Unemployment','CA'),('Unemployment','MI'),('Unemployment','PA'),
    ('HouseIdx','CA'),('HouseIdx','MI'),('HouseIdx','PA'),
    ('MedIncome','CA'),('MedIncome','MI'),('MedIncome','PA')
    ])
print("\n\n DATA AFTER FORMATTING: \n\n")
print(macro_data[:20]) # this is a dataset that is "wide", and now 
                       # the column variable names have 2 levels - var name, 
                       # and unit/state that variable applies to




 DATA BEFORE FORMATTING: 


      CAUR  MIUR  PAUR      LXXRSA      DEXRSA      WDXRSA  MEHOINUSCAA672N  \
DATE                                                                          
1990   5.2   7.8   5.1  100.471193         NaN   93.362855          63333.0   
1991   7.1   8.8   6.6   95.569015   58.420806   89.706871          61797.0   
1992   8.5   9.4   7.5   92.786926   59.748947   88.573807          62517.0   
1993   9.7   7.6   7.3   85.246295   61.564205   89.065118          59529.0   
1994   9.2   7.1   6.6   77.395052   64.526663   88.988467          60464.0   
1995   7.7   5.2   5.8   76.376389   68.973289   89.670303          61846.0   
1996   7.7   4.9   5.9   73.919989   73.582339   88.655369          63176.0   
1997   6.8   4.7   5.3   74.536884   79.347642   88.775224          63245.0   
1998   6.0   4.0   4.7   81.035037   85.183613   90.108149          64349.0   
1999   5.6   3.9   4.5   92.140086   92.433567   94.188054          67156.0   
2000   5.0   3.3   4.1

- Q6: for each decade and state, report the average annual CHANGE (level, not percent) in unemployment
- Q7: for each decade and state, report the average annual PERCENT CHANGE in house prices and household income

In [16]:
# do your work here

(macro_data

 # Q6
    # create decade variable 
     .reset_index()
     .assign(decade = lambda x: (x['DATE']//10)*10)
 
    # make sure we cna access states
 
    # create var: change in unemployement (subtract from prior year, not % change!)
     
    # avergae unemployment for each state with decade

)
   



DATE Unemployment               HouseIdx                          \
                   CA    MI   PA          CA          MI          PA   
0   1990          5.2   7.8  5.1  100.471193         NaN   93.362855   
1   1991          7.1   8.8  6.6   95.569015   58.420806   89.706871   
2   1992          8.5   9.4  7.5   92.786926   59.748947   88.573807   
3   1993          9.7   7.6  7.3   85.246295   61.564205   89.065118   
4   1994          9.2   7.1  6.6   77.395052   64.526663   88.988467   
5   1995          7.7   5.2  5.8   76.376389   68.973289   89.670303   
6   1996          7.7   4.9  5.9   73.919989   73.582339   88.655369   
7   1997          6.8   4.7  5.3   74.536884   79.347642   88.775224   
8   1998          6.0   4.0  4.7   81.035037   85.183613   90.108149   
9   1999          5.6   3.9  4.5   92.140086   92.433567   94.188054   
10  2000          5.0   3.3  4.1  101.031209  100.145625  100.871832   
11  2001          4.8   4.4  4.3  112.031476  107.327208  113.794411   
12  2002          6.5   6.4  5.5  122.706507  111.468549  127.245701   
13  2003          6.8   6.6  5.8  145.752401  115.704198  146.676851   
14  2004          6.5   6.8  5.5  178.773752  119.772720  169.535303   
15  2005          5.8   7.2  5.2  221.471364  123.264823  210.799937   
16  2006          5.0   6.8  4.7  268.208844  126.872678  250.272528   
17  2007          4.9   6.9  4.3  270.804920  118.163891  241.411126   
18  2008          6.0   7.1  4.7  226.111966  100.378575  215.656581   
19  2009          9.7  10.9  7.0  167.831216   77.690858  174.183926   
20  2010         12.2  13.9  8.7  174.441411   72.140927  179.971213   
21  2011         12.1  11.0  7.9  171.509501   68.064811  184.312385   
22  2012         11.0   9.2  7.7  162.454660   70.920882  180.074712   
23  2013          9.6   9.1  7.9  182.187354   82.235845  190.881554   
24  2014          8.2   8.1  6.5  216.523446   95.261215  206.946317   
25  2015          6.8   6.0  5.4  228.186011   98.246165  208.677978   
26  2016          5.7   4.9  5.2  243.376558  104.923573  212.061335   
27  2017          5.2   4.9  5.2  256.068405  111.041448  218.168329   
28  2018          4.4   4.5  4.6  274.883740  119.371453  223.047876   

   MedIncome                   decade  
          CA       MI       PA         
0    63333.0  56954.0  55181.0   1990  
1    61797.0  58957.0  55744.0   1990  
2    62517.0  57795.0  53523.0   1990  
3    59529.0  57064.0  54151.0   1990  
4    60464.0  60384.0  54877.0   1990  
5    61846.0  60872.0  57693.0   1990  
6    63176.0  63849.0  56807.0   1990  
7    63245.0  61728.0  59776.0   1990  
8    64349.0  65744.0  61333.0   1990  
9    67156.0  70942.0  58119.0   1990  
10   69696.0  67755.0  62789.0   2000  
11   68413.0  65206.0  62966.0   2000  
12   67600.0  60871.0  60562.0   2000  
13   68695.0  62734.0  59823.0   2000  
14   66782.0  57331.0  59841.0   2000  
15   67918.0  60278.0  60760.0   2000  
16   70316.0  61835.0  61619.0   2000  
17   68890.0  61024.0  59870.0   2000  
18   67865.0  59264.0  61185.0   2000  
19   67051.0  54939.0  57540.0   2000  
20   63788.0  54379.0  56773.0   2010  
21   60795.0  55682.0  56857.0   2010  
22   63609.0  55795.0  57902.0   2010  
23   66829.0  62182.0  60631.0   2010  
24   65384.0  56216.0  59640.0   2010  
25   68670.0  58491.0  65166.0   2010  
26   70993.0  60823.0  64965.0   2010  
27   73045.0  58827.0  63916.0   2010  
28   71766.0  61544.0  65693.0   2010